# Prompt Tuning - NL2SQL using Santacoder

### Load dataset

In [1]:
from datasets import load_dataset

dataset = load_dataset("wikisql")

/home/logesh_umapathi/miniconda3/envs/llama_4bit/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset wikisql (/mnt/data/logesh/hf_cache/wikisql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d)
100%|██████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 353.15it/s]


In [60]:
dataset['train'][0]

{'phase': 1,
 'question': 'Tell me what the notes are for South Australia ',
 'table': {'header': ['State/territory',
   'Text/background colour',
   'Format',
   'Current slogan',
   'Current series',
   'Notes'],
  'page_title': '',
  'page_id': '',
  'types': ['text', 'text', 'text', 'text', 'text', 'text'],
  'id': '1-1000181-1',
  'section_title': '',
  'caption': '',
  'rows': [['Australian Capital Territory',
    'blue/white',
    'Yaa·nna',
    'ACT · CELEBRATION OF A CENTURY 2013',
    'YIL·00A',
    'Slogan screenprinted on plate'],
   ['New South Wales',
    'black/yellow',
    'aa·nn·aa',
    'NEW SOUTH WALES',
    'BX·99·HI',
    'No slogan on current series'],
   ['New South Wales',
    'black/white',
    'aaa·nna',
    'NSW',
    'CPX·12A',
    'Optional white slimline series'],
   ['Northern Territory',
    'ochre/white',
    'Ca·nn·aa',
    'NT · OUTBACK AUSTRALIA',
    'CB·06·ZZ',
    'New series began in June 2011'],
   ['Queensland',
    'maroon/white',
    'nnn·aaa

## Initialize model

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm import tqdm

device = "cuda"
model_name_or_path = "bigcode/santacoder"
tokenizer_name_or_path = "bigcode/santacoder"
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=8,
    prompt_tuning_init_text="Convert question in natural language to SQL",
    tokenizer_name_or_path=model_name_or_path,
)


text_column = "question"
label_column = "human_readable"
max_length = 64
lr = 3e-2
num_epochs = 3
batch_size = 8

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

## Converting wikisql to instruction dataset

```Format: Question : <Question> \n Table Columns : <List_of_columns> \n SQL : <SQL_TO_BE_GENERATED>```

In [7]:
def fetch_table_context(table):
    header_type = [f"{header}:{typ}" for header,typ in zip(table['header'],table['types'])]
    return ",".join(header_type)


def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} \n Table Columns : {fetch_table_context(t)} \n SQL : " for x,t in zip(examples[text_column],examples['table'])] 
    print(inputs[0])
    targets = [str(x[label_column])+'\n' for x in examples["sql"]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
train_processed_datasets = dataset['train'].map(
    preprocess_function,
    batched=True,
    num_proc=1,
    load_from_cache_file=False,
    remove_columns=dataset["train"].column_names,
    desc="Running tokenizer on dataset",
)

test_processed_datasets = dataset['test'].map(
    preprocess_function,
    batched=True,
    num_proc=1,
    load_from_cache_file=False,
    remove_columns=dataset["test"].column_names,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   2%|▍                        | 1000/56355 [00:00<00:15, 3642.70 examples/s]

question : Tell me what the notes are for South Australia  
 Table Columns : State/territory:text,Text/background colour:text,Format:text,Current slogan:text,Current series:text,Notes:text 
 SQL : 
question : What is the number for years 1985-88 
 Table Columns : Player:text,No.:real,Nationality:text,Position:text,Years for Jazz:text,School/Club Team:text 
 SQL : 


Running tokenizer on dataset:   5%|█▎                       | 3000/56355 [00:00<00:14, 3586.90 examples/s]

question : What is the barrel length for a cold model le6921sp? 
 Table Columns : Colt model no.:text,Name:text,Stock:text,Fire control:text,Rear sight:text,Forward assist:text,Barrel length:text,Barrel profile:text,Barrel twist:text,Hand guards:text,Bayonet Lug:text,Muzzle device:text 
 SQL : 


Running tokenizer on dataset:   7%|█▊                       | 4000/56355 [00:01<00:13, 3753.78 examples/s]

...

Running tokenizer on dataset:  88%|█████████████████████▏  | 14000/15878 [00:04<00:00, 3304.89 examples/s]

question : What country is Steve Jones from? 
 Table Columns : Place:text,Player:text,Country:text,Score:text,To par:text,Money ( $ ):real 
 SQL : 


Running tokenizer on dataset:  94%|██████████████████████▋ | 15000/15878 [00:04<00:00, 3370.49 examples/s]

question : What is the lowest number of f/laps with more than 4 podiums and more than 14 races? 
 Table Columns : Season:real,Series:text,Team:text,Races:real,Wins:real,Poles:real,F/Laps:real,Podiums:real,Points:real,Position:text 
 SQL : 


question : What Golden point(s) scorer has the Away Brisbane Broncos and Home South Sydney Rabbitohs? 
 Table Columns : Home:text,Score:text,Away:text,Venue:text,Golden point(s) scorer:text 
 SQL : 


In [9]:
train_dataloader = DataLoader(
    train_processed_datasets, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(test_processed_datasets, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

## Train

In [10]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,trust_remote_code=True)
model = get_peft_model(model, peft_config)
print(model.print_trainable_parameters())

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


trainable params: 16384 || all params: 1124902912 || trainable%: 0.0014564812505348018
None


In [11]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [12]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|█████████████████████████████████████████████████████████████████| 1985/1985 [05:24<00:00,  6.11it/s]


epoch=0: train_ppl=tensor(1.3023, device='cuda:0') train_epoch_loss=tensor(0.2641, device='cuda:0') eval_ppl=tensor(nan, device='cuda:0') eval_epoch_loss=tensor(nan, device='cuda:0')


100%|████████████████████████████████████████████████████████████| 1985/1985 [05:27<00:00,  6.07it/s]


epoch=1: train_ppl=tensor(nan, device='cuda:0') train_epoch_loss=tensor(nan, device='cuda:0') eval_ppl=tensor(nan, device='cuda:0') eval_epoch_loss=tensor(nan, device='cuda:0')


100%|████████████████████████████████████████████████████████████| 1985/1985 [05:26<00:00,  6.08it/s]

epoch=2: train_ppl=tensor(nan, device='cuda:0') train_epoch_loss=tensor(nan, device='cuda:0') eval_ppl=tensor(nan, device='cuda:0') eval_epoch_loss=tensor(nan, device='cuda:0')


In [13]:
model.save_pretrained("./prompt_model")


### Inference Test

In [57]:

dataset['test'][40]['question'], fetch_table_context(dataset['test'][40]['table'])

('Which Frequency is used for WEGP calls?',
 'Calls:text,Frequency:text,Branding:text,Format:text,Market/Rank:text,Timeslot:text,Group owner:text')

In [58]:
def infer(model,input_text):
    inputs = tokenizer(input_text,return_tensors="pt")
    with torch.no_grad():
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model.generate(
            input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=40, eos_token_id=stop_words_ids[0])
    
    return tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)

def parse(text):
    return text.split("SQL :")[1]

In [59]:
input_text = """question : What is the average score of students in maths 
 Table Columns : id:INT,name:text,subject:Text,score:INT
 SQL : """
 
predictions = infer(model,input_text)
print(parse(predictions[0]))

Setting `pad_token_id` to `eos_token_id`:185 for open-end generation.


 SELECT AVG score FROM table WHERE subject = maths



question : What is the average score of students in maths 
 Table Columns : id:INT,name:text,subject:Text,score:INT
 SQL : SELECT AVG score FROM table WHERE subject = maths
t.println(result
